# Segmenting and Clustering Neighborhoods in Toronto
### from Michael Moxey 

In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Scraping Wikipedia for postal code information table 

In [2]:
#importing libraries to help me scrape 
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen



# Using Beatiful soup to parse through wikipedia webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html,'html.parser')

In [3]:
# Focusing down on the table information
tables = soup.find_all('table',{'class':'wikitable sortable'})

In [33]:
# Displaying the table to identify the parts of the table
#tables

In [5]:
# Creating Array variables to hold the table information
postalCodes = []
boroughs = []
neighborhoods = []

# this loop will iterate throughout the table and place all of the cells into their appropriate columns
for table in tables:
    rows = table.find_all('tr')

    for row in rows:
        
        cells = row.find_all('td')
       
        
        if len(cells) > 1:
            
            postalCode = cells[0]
            postalCodes.append(postalCode.text.strip())
            
            borough = cells[1]
            boroughs.append(borough.text.strip())
            
            neighborhood = cells[2]
            neighborhoods.append(neighborhood.text.strip())


## Cleaning the Data

In [22]:
#Creating a dataframe to better manipulate the data
df1 = pd.DataFrame(postalCodes, columns = ['Postal Code'])
df1['Borough'] = boroughs 
df1['Neighborhoods']= neighborhoods

#Displaying the fully developed dataframe
df1.head(10)

,Postal Code,Borough,Neighborhoods
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [24]:
#checking the length making sure all data has been collected
print("the length of the borough Postal codes and Neighborhood are " +str(len(df1['Borough']))+", "+str(len(df1['Postal Code']))+", "+str(len(df1['Neighborhoods'])))

the length of the borough Postal codes and Neighborhood are 180, 180, 180


In [25]:
#replacing not "not assigned" with NAN to make further computations easier
df1.replace('Not assigned',np.NAN,inplace =True)


In [26]:
# dropping all cells with no boroughs
df1.dropna(how ='any',subset=['Borough'],inplace =True)
df1.shape

(103, 3)

In [27]:
#replacing the Neighborhoods that are NAN with the borough
df1.replace( np.NAN ,df1['Borough'],inplace = True)
print(df1.shape)

(103, 3)


In [28]:
# reading Longitude and latitude
df2 = pd.read_csv (r'C:\Users\micha\Downloads\Geospatial_Coordinates.csv')
print (df2.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [65]:
# Making the column on which we are joing are the same data type
df1=df1.astype({'Postal Code':str})
df2=df2.astype({'Postal Code':str})
df2.dtypes

#merging the data left join
df = df1.merge(df2, on='Postal Code', how='left')
df.head()

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
